In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from statsmodels.tsa.statespace.sarimax import SARIMAX
import os
from openpyxl import load_workbook
from openpyxl.drawing.image import Image

In [ ]:
!pip install openpyxl

In [16]:
# ----------------------------
# CONFIGURATION
# ----------------------------
run_forecast = True
run_churn = True

forecast_path = "client_forecast.csv"
churn_path = "client_churn.csv"
output_dir = "outputs"
os.makedirs(output_dir, exist_ok=True)

insights_list = []

In [17]:
# ----------------------------
# VALIDATION FUNCTIONS
# ----------------------------
def validate_forecast_csv(path):
    df = pd.read_csv(path)
    if 'date' not in df.columns or 'value' not in df.columns:
        raise ValueError("Forecast CSV must have 'date' and 'value' columns.")
    if df['date'].isnull().any() or df['value'].isnull().any():
        raise ValueError("No missing values allowed in 'date' or 'value'.")
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    if df['date'].isnull().any():
        raise ValueError("Invalid date format detected. Use YYYY-MM-DD.")
    print("Forecast CSV validated successfully.")
    return df

def validate_churn_csv(path):
    df = pd.read_csv(path)
    required = ['customer_id', 'churn']
    for col in required:
        if col not in df.columns:
            raise ValueError(f"Churn CSV must have '{col}' column.")
    if df['customer_id'].isnull().any() or df['churn'].isnull().any():
        raise ValueError("No missing values allowed in 'customer_id' or 'churn'.")
    print("Churn CSV validated successfully.")
    return df

In [ ]:
# ----------------------------
# FORECAST MODEL
# ----------------------------
if run_forecast:
    df_forecast = validate_forecast_csv(forecast_path)
    df_forecast = df_forecast.sort_values('date').set_index('date')

    model = SARIMAX(
        df_forecast['value'],
        order=(1,1,1),
        seasonal_order=(0,0,0,0),
        enforce_stationarity=False,
        enforce_invertibility=False
    )
    results = model.fit(disp=False)

    forecast_steps = 30
    forecast = results.get_forecast(steps=forecast_steps)
    forecast_df = forecast.summary_frame()
    forecast_df.to_csv(f"{output_dir}/predictions_forecast.csv")

    # forecast chart
    plt.figure(figsize=(10,6))
    plt.plot(df_forecast.index, df_forecast['value'], label='Actual', color='blue', linewidth=2, marker='o')
    plt.plot(forecast_df.index, forecast_df['mean'], label='Forecast', color='orange', linewidth=2, linestyle='--')
    plt.fill_between(forecast_df.index, forecast_df['mean_ci_lower'], forecast_df['mean_ci_upper'], color='orange', alpha=0.2, label='Confidence Interval')

    # Trend annotation
    last_actual = df_forecast['value'].iloc[-1]
    predicted_mean = forecast_df['mean'].mean()
    pct_change = (predicted_mean - last_actual) / last_actual * 100
    trend_text = f"Expected {pct_change:.1f}% {'increase' if pct_change>0 else 'decrease'} next month"
    plt.annotate(trend_text, xy=(forecast_df.index[-1], forecast_df['mean'].iloc[-1]), xytext=(-150,30),
                 textcoords='offset points', fontsize=10, color='green',
                 arrowprops=dict(facecolor='green', shrink=0.05))

    plt.title('Sales Forecast', fontsize=16)
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Value', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    forecast_chart_path = f"{output_dir}/forecast_chart.png"
    plt.savefig(forecast_chart_path, dpi=300)
    plt.close()

    # Insights
    trend = "increase" if pct_change > 0 else "decrease"
    insights_list.append(
        f"Forecast Insight:\n- Last recorded value: {last_actual:.0f}\n"
        f"- Predicted average next {forecast_steps} steps: {predicted_mean:.0f} ({pct_change:.1f}% {trend})\n"
        f"- Forecast confidence interval: {forecast_df['mean_ci_lower'].min():.0f} - {forecast_df['mean_ci_upper'].max():.0f}\n"
    )


Forecast CSV validated successfully.


c:\Users\mohse\anaconda3\envs\gymenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency ME will be used.
  self._init_dates(dates, freq)
c:\Users\mohse\anaconda3\envs\gymenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency ME will be used.
  self._init_dates(dates, freq)


In [ ]:
# ----------------------------
# CHURN MODEL
# ----------------------------
if run_churn:
    df_churn = validate_churn_csv(churn_path)

    X = df_churn.drop(columns=['customer_id','churn'])
    y = df_churn['churn']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)

    df_churn['churn_probability'] = model.predict_proba(X)[:,1]
    df_churn['risk_level'] = pd.cut(
        df_churn['churn_probability'],
        bins=[0,0.3,0.6,1.0],
        labels=['Low','Medium','High']
    )
    df_churn.to_csv(f"{output_dir}/predictions_churn.csv", index=False)

        # churn chart
    risk_counts = df_churn['risk_level'].value_counts().reindex(['Low','Medium','High']).fillna(0)
    risk_percent = (risk_counts / len(df_churn) * 100).round(1)

    plt.figure(figsize=(8,6))
    bars = plt.bar(risk_counts.index, risk_counts.values, color=['green','orange','red'], alpha=0.8)
    for bar, pct in zip(bars, risk_percent):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height()+0.5, f'{pct}%', ha='center', fontsize=10)
    plt.title('Customer Churn Risk Distribution', fontsize=16)
    plt.xlabel('Risk Level', fontsize=12)
    plt.ylabel('Number of Customers', fontsize=12)
    plt.grid(axis='y', linestyle='--', alpha=0.5)
    plt.tight_layout()
    churn_chart_path = f"{output_dir}/churn_chart.png"
    plt.savefig(churn_chart_path, dpi=300)
    plt.close()

    total_customers = len(df_churn)
    high_risk = df_churn[df_churn['risk_level']=='High'].shape[0]
    medium_risk = df_churn[df_churn['risk_level']=='Medium'].shape[0]

    insights_list.append(
        f"Churn Insight:\n- Total customers analyzed: {total_customers}\n"
        f"- High-risk customers: {high_risk} ({high_risk/total_customers*100:.1f}%)\n"
        f"- Medium-risk customers: {medium_risk} ({medium_risk/total_customers*100:.1f}%)\n"
        f"- Focus retention efforts on high-risk group.\n"
    )


Churn CSV validated successfully.


In [ ]:
# ----------------------------
# Export insights
# ----------------------------
with open(f"{output_dir}/insights.txt", "w") as f:
    f.write("\n".join(insights_list))

# ----------------------------
# Excel Dashboard with charts
# ----------------------------
with pd.ExcelWriter(f"{output_dir}/dashboard.xlsx", engine='openpyxl') as writer:
    if run_forecast:
        df_forecast.reset_index().to_excel(writer, sheet_name="Forecast_Data", index=False)
        forecast_df.reset_index().to_excel(writer, sheet_name="Forecast_Predictions", index=False)
    if run_churn:
        df_churn.to_excel(writer, sheet_name="Churn_Data", index=False)
    pd.DataFrame({"Insights": insights_list}).to_excel(writer, sheet_name="Insights", index=False)


# Insert charts using openpyxl
wb = load_workbook(f"{output_dir}/dashboard.xlsx")
if run_forecast:
    ws_forecast = wb['Forecast_Predictions']
    img_forecast = Image(forecast_chart_path)
    img_forecast.anchor = 'G2'
    ws_forecast.add_image(img_forecast)

if run_churn:
    ws_churn = wb['Churn_Data']
    img_churn = Image(churn_chart_path)
    img_churn.anchor = 'G2'
    ws_churn.add_image(img_churn)

wb.save(f"{output_dir}/dashboard.xlsx")

print("✅ MVP Productized framework executed successfully charts!")


✅ MVP Productized framework executed successfully with polished charts!
